In [1]:
import pandas as pd
import keras
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, BatchNormalization,Input
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import regularizers,initializers
from keras.layers import GaussianNoise
import keras.backend as Kb
import keras.losses

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import class_weight
import numpy as np
from numpy import exp
# Library for Gaussian process
# import GPy
##Library for visualization
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib;matplotlib.rcParams['figure.figsize'] = (8,6)
import pylab 
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
# test case with one dataset
df_loc = pd.read_csv("../synthetic_data_simulations/2d_gaussian_1200_"+str(1)+".csv",sep = ",")
# df_loc = pd.read_csv("../2D_biv_matern_large.csv", sep = ",")

In [3]:
df_loc.head()

,Unnamed: 0,x,y,var1,var2
0,1,0.000000,0.0,-0.312110,0.020474
1,2,0.025641,0.0,-0.327851,0.282155
2,3,0.051282,0.0,-0.127759,0.479609
3,4,0.076923,0.0,0.134564,0.674018
4,5,0.102564,0.0,0.324608,0.910878


In [4]:
N = len(df_loc)
s = np.vstack((df_loc["x"],df_loc["y"])).T
y = np.array(df_loc[["var1","var2"]])
# s_train, s_test, y_train, y_test= train_test_split(s, y, 
#                                                     test_size=0.17)
# s_test.shape

# N = len(s_test)
# s = s_test
# y= y_test

In [5]:
### Basis functions

num_basis = [5**2,7**2,9**2]
knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
##Wendland kernel
K = 0
phi = np.zeros((N, sum(num_basis)))
for res in range(len(num_basis)):
    theta = 1/np.sqrt(num_basis[res])*2.5
    knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
    knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
    for i in range(num_basis[res]):
        d = np.linalg.norm(s-knots[i,:],axis=1)/theta
        for j in range(len(d)):
            if d[j] >= 0 and d[j] <= 1:
                phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
            else:
                phi[j,i + K] = 0
    K = K + num_basis[res]

### Train Test split




In [6]:
phi.shape

(1200, 155)

In [38]:
### Basis functions

num_basis = [7**2,11**2,15**2]
knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
#knots_1d = [np.linspace(0,1,i) for i in num_basis]
##Wendland kernel
K = 0
phi = np.zeros((N, sum(num_basis)))
for res in range(len(num_basis)):
    theta = 1/np.sqrt(num_basis[res])*2.5
    knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
    knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
    for i in range(num_basis[res]):
        d = np.linalg.norm(s-knots[i,:],axis=1)/theta
        for j in range(len(d)):
            if d[j] >= 0 and d[j] <= 1:
                phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
            else:
                phi[j,i + K] = 0
    K = K + num_basis[res]

## Custom loss function

In [37]:
variance_var1 = np.var(df_loc["var1"])
variance_var2 = np.var(df_loc["var2"])


def custom_mse(y_true, y_pred):
 
    # calculating squared difference between target and predicted values 
    loss = Kb.square(y_pred - y_true)  # (batch_size, 2)
    
    # multiplying the values with weights along batch dimension
    loss = loss * [(1/variance_var1), (1/variance_var2)]          # (batch_size, 2)
                
    # summing both loss values along batch dimension 
    loss = Kb.sum(loss, axis=1)        # (batch_size,)
    
    return loss

In [38]:
variance_var2

0.9392859099082996

In [39]:
# testing the custom loss 
y_true = y[0:599,]
y_pred = y[600:1199,]

In [1]:
#custom_mse(y_true, y_pred)

## Replicates


In [49]:
## Functions for calculation of MSE and MAE
def mse(y_pred,y_true):
    mse = np.mean((y_pred-y_true)**2)
    return mse

def mae(y_pred,y_true):
    mae = np.mean(np.absolute(y_pred-y_true))
    return mae

mse_var1 = []
mse_var2 = []
for i in range(50):
    df_loc = pd.read_csv("../synthetic_data_simulations/2d_gaussian_1200_"+str(i+1)+".csv", sep = ",")
    
    
    variance_var1 = np.var(df_loc["var1"])
    variance_var2 = np.var(df_loc["var2"])


    def custom_mse(y_true, y_pred):

        # calculating squared difference between target and predicted values 
        loss = Kb.square(y_pred - y_true)  # (batch_size, 2)

        # multiplying the values with weights along batch dimension
        loss = loss * [(1/variance_var1), (1/variance_var2)]          # (batch_size, 2)

        # summing both loss values along batch dimension 
        loss = Kb.sum(loss, axis=1)        # (batch_size,)

        return loss
#     keras.losses.custom_loss = custom_mse

    N = len(df_loc)
    s = np.vstack((df_loc["x"],df_loc["y"])).T
    y = np.array(df_loc[["var1","var2"]])
    num_basis = [2**2,5**2,9**2]
    knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
    #knots_1d = [np.linspace(0,1,i) for i in num_basis]
    ##Wendland kernel
    K = 0
    phi = np.zeros((N, sum(num_basis)))
    for res in range(len(num_basis)):
        theta = 1/np.sqrt(num_basis[res])*2.5
        knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
        knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
        for i in range(num_basis[res]):
            d = np.linalg.norm(s-knots[i,:],axis=1)/theta
            for j in range(len(d)):
                if d[j] >= 0 and d[j] <= 1:
                    phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
                else:
                    phi[j,i + K] = 0
        K = K + num_basis[res]
    s_train, s_test, encoder_train, encoder_test    , y_train, y_test= train_test_split(s, phi, y, 
                                                                                    test_size=0.3333)
    N_train = s_train.shape[0]
    N_test = s_test.shape[0]

    
    # DeepKriging model for continuous data
    model = Sequential()
    # model.add(Dense(100, input_dim = 2,  kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(100, input_dim = encoder_train.shape[1],  
                kernel_initializer=initializers.RandomNormal(stddev=0.01), activation='relu'))
#     model.add(Dense(100, input_dim = encoder_train.shape[1],  kernel_initializer='he_uniform', activation='relu'))
    # model.add(Dropout(rate=0.5))
    # model.add(BatchNormalization())
#     model.add(Dense(100, activation='relu'))
    model.add(Dense(100, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
                bias_regularizer=regularizers.l2(1e-4),
                activity_regularizer=regularizers.l2(1e-5),activation='relu'))
#     model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
#     model.add(Dense(100, activation='relu'))
    model.add(Dense(50, activation='relu'))
    # model.add(Dense(100, activation='relu'))
    #model.add(Dropout(rate=0.5))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    #model.add(BatchNormalization())
    model.add(Dense(2, activation='linear'))
    NB_START_EPOCHS = 50 
    # NB_START_EPOCHS = 200  # Number of epochs we usually start to train with
    BATCH_SIZE = 64  
    fold_no = 1
    optimizer = keras.optimizers.Adam(lr=0.001)
    model.compile(optimizer=optimizer, loss=custom_mse, metrics=['mae','mse'])

    result = model.fit(encoder_train, y_train, 
                       validation_data=(encoder_test,y_test), epochs = 300, batch_size = 64, verbose = 2)
    
    callbacks = [EarlyStopping(monitor='val_loss', patience=40),
                 ModelCheckpoint(filepath='Biv_gaussian.h5', monitor='val_loss', save_best_only=True)]
    result = model.fit(encoder_train, y_train, callbacks=callbacks, 
                       validation_data=(encoder_test,y_test), epochs = 250, batch_size = 64, verbose = 2)
    model = keras.models.load_model('Biv_gaussian.h5', custom_objects={'custom_mse':custom_mse})
    y_pred = model.predict(encoder_test)


    # Mean Squared Error
    mse_var1.append(mse(y_pred[:,0], y_test[:,0]))
    mse_var2.append(mse(y_pred[:,1], y_test[:,1]))
    
# end_time = time.time()
# print("%s seconds", end_time - start_time)

Epoch 1/300
13/13 - 2s - loss: 2.6754 - mae: 0.7182 - mse: 1.3976 - val_loss: 2.4055 - val_mae: 0.6739 - val_mse: 1.2556
Epoch 2/300
13/13 - 0s - loss: 1.9574 - mae: 0.6164 - mse: 1.0490 - val_loss: 1.5446 - val_mae: 0.5472 - val_mse: 0.8346
Epoch 3/300
13/13 - 0s - loss: 1.4017 - mae: 0.4921 - mse: 0.7716 - val_loss: 1.0813 - val_mae: 0.4569 - val_mse: 0.5849
Epoch 4/300
13/13 - 0s - loss: 0.9039 - mae: 0.3981 - mse: 0.4947 - val_loss: 0.6779 - val_mae: 0.3770 - val_mse: 0.3662
Epoch 5/300
13/13 - 0s - loss: 0.5938 - mae: 0.3274 - mse: 0.3185 - val_loss: 0.5973 - val_mae: 0.3654 - val_mse: 0.3219
Epoch 6/300
13/13 - 0s - loss: 0.4990 - mae: 0.3147 - mse: 0.2640 - val_loss: 0.3867 - val_mae: 0.2767 - val_mse: 0.2006
Epoch 7/300
13/13 - 0s - loss: 0.3582 - mae: 0.2653 - mse: 0.1857 - val_loss: 0.3757 - val_mae: 0.2811 - val_mse: 0.1936
Epoch 8/300
13/13 - 0s - loss: 0.2925 - mae: 0.2392 - mse: 0.1477 - val_loss: 0.3130 - val_mae: 0.2546 - val_mse: 0.1597
Epoch 9/300
13/13 - 0s - loss: 0

Epoch 69/300
13/13 - 0s - loss: 0.0540 - mae: 0.0994 - mse: 0.0219 - val_loss: 0.1430 - val_mae: 0.1592 - val_mse: 0.0685
Epoch 70/300
13/13 - 0s - loss: 0.0582 - mae: 0.1029 - mse: 0.0241 - val_loss: 0.1416 - val_mae: 0.1632 - val_mse: 0.0679
Epoch 71/300
13/13 - 0s - loss: 0.0557 - mae: 0.1012 - mse: 0.0228 - val_loss: 0.1425 - val_mae: 0.1618 - val_mse: 0.0685
Epoch 72/300
13/13 - 0s - loss: 0.0611 - mae: 0.1078 - mse: 0.0255 - val_loss: 0.1497 - val_mae: 0.1621 - val_mse: 0.0718
Epoch 73/300
13/13 - 0s - loss: 0.0550 - mae: 0.0998 - mse: 0.0221 - val_loss: 0.1489 - val_mae: 0.1647 - val_mse: 0.0713
Epoch 74/300
13/13 - 0s - loss: 0.0604 - mae: 0.1043 - mse: 0.0251 - val_loss: 0.1434 - val_mae: 0.1610 - val_mse: 0.0692
Epoch 75/300
13/13 - 0s - loss: 0.0667 - mae: 0.1111 - mse: 0.0280 - val_loss: 0.1490 - val_mae: 0.1656 - val_mse: 0.0712
Epoch 76/300
13/13 - 0s - loss: 0.0581 - mae: 0.1057 - mse: 0.0238 - val_loss: 0.1712 - val_mae: 0.1689 - val_mse: 0.0827
Epoch 77/300
13/13 - 0s 

Epoch 136/300
13/13 - 0s - loss: 0.0415 - mae: 0.0846 - mse: 0.0156 - val_loss: 0.1297 - val_mae: 0.1528 - val_mse: 0.0619
Epoch 137/300
13/13 - 0s - loss: 0.0402 - mae: 0.0826 - mse: 0.0150 - val_loss: 0.1488 - val_mae: 0.1634 - val_mse: 0.0728
Epoch 138/300
13/13 - 0s - loss: 0.0419 - mae: 0.0842 - mse: 0.0161 - val_loss: 0.1473 - val_mae: 0.1570 - val_mse: 0.0712
Epoch 139/300
13/13 - 0s - loss: 0.0390 - mae: 0.0814 - mse: 0.0143 - val_loss: 0.1464 - val_mae: 0.1605 - val_mse: 0.0705
Epoch 140/300
13/13 - 0s - loss: 0.0389 - mae: 0.0801 - mse: 0.0143 - val_loss: 0.1491 - val_mae: 0.1598 - val_mse: 0.0731
Epoch 141/300
13/13 - 0s - loss: 0.0458 - mae: 0.0874 - mse: 0.0183 - val_loss: 0.1416 - val_mae: 0.1586 - val_mse: 0.0678
Epoch 142/300
13/13 - 0s - loss: 0.0421 - mae: 0.0857 - mse: 0.0160 - val_loss: 0.1422 - val_mae: 0.1631 - val_mse: 0.0687
Epoch 143/300
13/13 - 0s - loss: 0.0383 - mae: 0.0819 - mse: 0.0140 - val_loss: 0.1501 - val_mae: 0.1597 - val_mse: 0.0723
Epoch 144/300
13

Epoch 203/300
13/13 - 0s - loss: 0.0317 - mae: 0.0759 - mse: 0.0109 - val_loss: 0.1478 - val_mae: 0.1590 - val_mse: 0.0718
Epoch 204/300
13/13 - 0s - loss: 0.0318 - mae: 0.0735 - mse: 0.0110 - val_loss: 0.1450 - val_mae: 0.1562 - val_mse: 0.0706
Epoch 205/300
13/13 - 0s - loss: 0.0295 - mae: 0.0659 - mse: 0.0096 - val_loss: 0.1401 - val_mae: 0.1566 - val_mse: 0.0684
Epoch 206/300
13/13 - 0s - loss: 0.0283 - mae: 0.0661 - mse: 0.0090 - val_loss: 0.1334 - val_mae: 0.1521 - val_mse: 0.0645
Epoch 207/300
13/13 - 0s - loss: 0.0275 - mae: 0.0652 - mse: 0.0087 - val_loss: 0.1347 - val_mae: 0.1522 - val_mse: 0.0651
Epoch 208/300
13/13 - 0s - loss: 0.0257 - mae: 0.0609 - mse: 0.0078 - val_loss: 0.1331 - val_mae: 0.1513 - val_mse: 0.0646
Epoch 209/300
13/13 - 0s - loss: 0.0251 - mae: 0.0594 - mse: 0.0075 - val_loss: 0.1329 - val_mae: 0.1524 - val_mse: 0.0644
Epoch 210/300
13/13 - 0s - loss: 0.0245 - mae: 0.0578 - mse: 0.0072 - val_loss: 0.1356 - val_mae: 0.1521 - val_mse: 0.0659
Epoch 211/300
13

Epoch 270/300
13/13 - 0s - loss: 0.0246 - mae: 0.0602 - mse: 0.0077 - val_loss: 0.1457 - val_mae: 0.1613 - val_mse: 0.0721
Epoch 271/300
13/13 - 0s - loss: 0.0251 - mae: 0.0655 - mse: 0.0082 - val_loss: 0.1374 - val_mae: 0.1532 - val_mse: 0.0673
Epoch 272/300
13/13 - 0s - loss: 0.0235 - mae: 0.0588 - mse: 0.0073 - val_loss: 0.1348 - val_mae: 0.1548 - val_mse: 0.0660
Epoch 273/300
13/13 - 0s - loss: 0.0212 - mae: 0.0552 - mse: 0.0061 - val_loss: 0.1385 - val_mae: 0.1539 - val_mse: 0.0676
Epoch 274/300
13/13 - 0s - loss: 0.0218 - mae: 0.0552 - mse: 0.0064 - val_loss: 0.1374 - val_mae: 0.1540 - val_mse: 0.0667
Epoch 275/300
13/13 - 0s - loss: 0.0223 - mae: 0.0547 - mse: 0.0067 - val_loss: 0.1431 - val_mae: 0.1562 - val_mse: 0.0708
Epoch 276/300
13/13 - 0s - loss: 0.0248 - mae: 0.0601 - mse: 0.0079 - val_loss: 0.1386 - val_mae: 0.1579 - val_mse: 0.0680
Epoch 277/300
13/13 - 0s - loss: 0.0256 - mae: 0.0642 - mse: 0.0083 - val_loss: 0.1439 - val_mae: 0.1614 - val_mse: 0.0702
Epoch 278/300
13

...

Epoch 38/300
13/13 - 0s - loss: 0.1157 - mae: 0.1574 - mse: 0.0531 - val_loss: 0.1623 - val_mae: 0.1767 - val_mse: 0.0762
Epoch 39/300
13/13 - 0s - loss: 0.0981 - mae: 0.1376 - mse: 0.0429 - val_loss: 0.1605 - val_mae: 0.1826 - val_mse: 0.0760
Epoch 40/300
13/13 - 0s - loss: 0.0854 - mae: 0.1312 - mse: 0.0363 - val_loss: 0.1619 - val_mae: 0.1795 - val_mse: 0.0759
Epoch 41/300
13/13 - 0s - loss: 0.0891 - mae: 0.1320 - mse: 0.0383 - val_loss: 0.1606 - val_mae: 0.1796 - val_mse: 0.0760
Epoch 42/300
13/13 - 0s - loss: 0.0814 - mae: 0.1270 - mse: 0.0342 - val_loss: 0.1629 - val_mae: 0.1789 - val_mse: 0.0766
Epoch 43/300
13/13 - 0s - loss: 0.0814 - mae: 0.1250 - mse: 0.0346 - val_loss: 0.1524 - val_mae: 0.1743 - val_mse: 0.0719
Epoch 44/300
13/13 - 0s - loss: 0.0869 - mae: 0.1298 - mse: 0.0373 - val_loss: 0.1542 - val_mae: 0.1728 - val_mse: 0.0729
Epoch 45/300
13/13 - 0s - loss: 0.1008 - mae: 0.1383 - mse: 0.0449 - val_loss: 0.1741 - val_mae: 0.1928 - val_mse: 0.0838
Epoch 46/300
13/13 - 0s 

13/13 - 0s - loss: 0.0469 - mae: 0.0891 - mse: 0.0172 - val_loss: 0.1413 - val_mae: 0.1632 - val_mse: 0.0664
Epoch 106/300
13/13 - 0s - loss: 0.0531 - mae: 0.0955 - mse: 0.0209 - val_loss: 0.1443 - val_mae: 0.1651 - val_mse: 0.0686
Epoch 107/300
13/13 - 0s - loss: 0.0513 - mae: 0.0937 - mse: 0.0192 - val_loss: 0.1646 - val_mae: 0.1682 - val_mse: 0.0784
Epoch 108/300
13/13 - 0s - loss: 0.0504 - mae: 0.0955 - mse: 0.0190 - val_loss: 0.1485 - val_mae: 0.1647 - val_mse: 0.0707
Epoch 109/300
13/13 - 0s - loss: 0.0500 - mae: 0.0967 - mse: 0.0188 - val_loss: 0.1372 - val_mae: 0.1635 - val_mse: 0.0647
Epoch 110/300
13/13 - 0s - loss: 0.0498 - mae: 0.0941 - mse: 0.0188 - val_loss: 0.1505 - val_mae: 0.1736 - val_mse: 0.0719
Epoch 111/300
13/13 - 0s - loss: 0.0555 - mae: 0.0993 - mse: 0.0223 - val_loss: 0.1640 - val_mae: 0.1837 - val_mse: 0.0797
Epoch 112/300
13/13 - 0s - loss: 0.0631 - mae: 0.1110 - mse: 0.0263 - val_loss: 0.1650 - val_mae: 0.1782 - val_mse: 0.0786
Epoch 113/300
13/13 - 0s - los

Epoch 172/300
13/13 - 0s - loss: 0.0409 - mae: 0.0835 - mse: 0.0144 - val_loss: 0.1373 - val_mae: 0.1589 - val_mse: 0.0656
Epoch 173/300
13/13 - 0s - loss: 0.0377 - mae: 0.0793 - mse: 0.0130 - val_loss: 0.1453 - val_mae: 0.1624 - val_mse: 0.0693
Epoch 174/300
13/13 - 0s - loss: 0.0359 - mae: 0.0754 - mse: 0.0120 - val_loss: 0.1441 - val_mae: 0.1627 - val_mse: 0.0694
Epoch 175/300
13/13 - 0s - loss: 0.0399 - mae: 0.0813 - mse: 0.0145 - val_loss: 0.1649 - val_mae: 0.1615 - val_mse: 0.0798
Epoch 176/300
13/13 - 0s - loss: 0.0443 - mae: 0.0886 - mse: 0.0170 - val_loss: 0.1400 - val_mae: 0.1685 - val_mse: 0.0674
Epoch 177/300
13/13 - 0s - loss: 0.0453 - mae: 0.0883 - mse: 0.0169 - val_loss: 0.1477 - val_mae: 0.1636 - val_mse: 0.0703
Epoch 178/300
13/13 - 0s - loss: 0.0444 - mae: 0.0854 - mse: 0.0171 - val_loss: 0.1534 - val_mae: 0.1616 - val_mse: 0.0740
Epoch 179/300
13/13 - 0s - loss: 0.0494 - mae: 0.0953 - mse: 0.0198 - val_loss: 0.1426 - val_mae: 0.1643 - val_mse: 0.0681
Epoch 180/300
13

Epoch 239/300
13/13 - 0s - loss: 0.0262 - mae: 0.0598 - mse: 0.0078 - val_loss: 0.1313 - val_mae: 0.1548 - val_mse: 0.0630
Epoch 240/300
13/13 - 0s - loss: 0.0250 - mae: 0.0590 - mse: 0.0071 - val_loss: 0.1407 - val_mae: 0.1568 - val_mse: 0.0683
Epoch 241/300
13/13 - 0s - loss: 0.0252 - mae: 0.0596 - mse: 0.0073 - val_loss: 0.1363 - val_mae: 0.1550 - val_mse: 0.0660
Epoch 242/300
13/13 - 0s - loss: 0.0279 - mae: 0.0659 - mse: 0.0088 - val_loss: 0.1303 - val_mae: 0.1533 - val_mse: 0.0625
Epoch 243/300
13/13 - 0s - loss: 0.0279 - mae: 0.0659 - mse: 0.0086 - val_loss: 0.1384 - val_mae: 0.1570 - val_mse: 0.0672
Epoch 244/300
13/13 - 0s - loss: 0.0248 - mae: 0.0592 - mse: 0.0070 - val_loss: 0.1370 - val_mae: 0.1553 - val_mse: 0.0661
Epoch 245/300
13/13 - 0s - loss: 0.0233 - mae: 0.0559 - mse: 0.0061 - val_loss: 0.1361 - val_mae: 0.1544 - val_mse: 0.0654
Epoch 246/300
13/13 - 0s - loss: 0.0254 - mae: 0.0597 - mse: 0.0074 - val_loss: 0.1361 - val_mae: 0.1571 - val_mse: 0.0653
Epoch 247/300
13

Epoch 6/250
13/13 - 0s - loss: 0.0252 - mae: 0.0611 - mse: 0.0076 - val_loss: 0.1532 - val_mae: 0.1663 - val_mse: 0.0743
Epoch 7/250
13/13 - 0s - loss: 0.0316 - mae: 0.0707 - mse: 0.0107 - val_loss: 0.1432 - val_mae: 0.1666 - val_mse: 0.0702
Epoch 8/250
13/13 - 0s - loss: 0.0295 - mae: 0.0681 - mse: 0.0098 - val_loss: 0.1307 - val_mae: 0.1552 - val_mse: 0.0630
Epoch 9/250
13/13 - 0s - loss: 0.0252 - mae: 0.0607 - mse: 0.0076 - val_loss: 0.1529 - val_mae: 0.1621 - val_mse: 0.0744
Epoch 10/250
13/13 - 0s - loss: 0.0256 - mae: 0.0629 - mse: 0.0079 - val_loss: 0.1324 - val_mae: 0.1561 - val_mse: 0.0645
Epoch 11/250
13/13 - 0s - loss: 0.0237 - mae: 0.0576 - mse: 0.0069 - val_loss: 0.1277 - val_mae: 0.1516 - val_mse: 0.0620
Epoch 12/250
13/13 - 0s - loss: 0.0217 - mae: 0.0539 - mse: 0.0059 - val_loss: 0.1336 - val_mae: 0.1551 - val_mse: 0.0651
Epoch 13/250
13/13 - 0s - loss: 0.0210 - mae: 0.0518 - mse: 0.0056 - val_loss: 0.1405 - val_mae: 0.1542 - val_mse: 0.0688
Epoch 14/250
13/13 - 0s - lo

In [13]:
mse_var1_old = mse_var1.copy()
mse_var2_old = mse_var2.copy()

In [56]:
np.mean(mse_var1_old)


0.0597319327165765

In [58]:
np.mean(mse_var1)

0.05721901860533482

In [123]:
print("mean var 1 : ",np.mean(mse_var1))
print("mean var 2 : ",np.mean(mse_var2))
print("variance var 1 : ",np.var(mse_var1))
print("variance var 2 : ",np.var(mse_var2))
print("correlation : ",np.mean(correlation))

mean var 1 :  0.0064385244491971284
mean var 2 :  0.00628816114063689
variance var 1 :  7.806350819686705e-07
variance var 2 :  7.948728898839889e-07
correlation :  0.7117337666834473


In [60]:
df_mse = pd.DataFrame(np.vstack((mse_var1,mse_var2)).T, columns = ["mse_var1","mse_var2"])

df_mse.to_csv("../plot_results/DeepKriging_gaussian_mse.csv")

mean var 1 :  0.19668477192442524
mean var 2 :  0.006429057269861566
variance var 1 :  0.0
variance var 2 :  0.0

## saving the dataset

In [179]:
df_train_loc = pd.DataFrame(s_train, columns = ["x","y"])
df_train_val = pd.DataFrame(y_train, columns = ["var1","var2"])
df_train = pd.concat([df_train_loc,df_train_val], axis = 1)

In [180]:
df_train.to_csv("2d_biv_matern_training_1200.csv")

In [181]:
df_test_loc = pd.DataFrame(s_test, columns = ["x","y"])
df_test_val = pd.DataFrame(y_test, columns = ["var1","var2"])
df_test = pd.concat([df_test_loc,df_test_val], axis = 1)
df_test.to_csv("2d_biv_matern_testing_1200.csv")

In [29]:
df_loc = pd.DataFrame(s,columns = ["x","y"])
df_val = pd.DataFrame(y, columns = ["var1","var2"])
df = pd.concat([df_loc,df_val], axis = 1)
df.to_csv("2d_biv_gaussian_full.csv")

In [349]:
# save generated data
df_loc = pd.DataFrame(X,columns = ["x","y","var1","var2"])
df_loc.to_csv("generated_data.csv")